load data
根据原论文内容，要：
transform the .mrc file into np array
chunked into pairs of overlapping boxes of size 60*60*60 with strides of 30 voxels
augmentation:
random 90 degree rotation
randomly cropping 48*48*48 box from 60*60*60box

In [24]:
import os
import mrcfile
import numpy as np
import interp_back
import torch
import torchvision
from torch.utils import data
from torch.utils.data import Dataset
from torchvision import transforms
from utils import pad_map, chunk_generator, parse_map

depoFolder = "/home/tyche/training_and_validation_sets/depoFiles"
simuFolder = "/home/tyche/training_and_validation_sets/simuFiles"



In [25]:
def split_map_into_overlapped_chunk(map, box_size, stride, dtype=np.float32, padding=0.0):
    assert stride <= box_size
    map_shape = np.shape(map)
    padded_map = np.full((map_shape[0] + 2 * box_size, map_shape[1] + 2 * box_size, map_shape[2] + 2 * box_size), padding, dtype=dtype)
    padded_map[box_size : box_size + map_shape[0], box_size : box_size + map_shape[1], box_size : box_size + map_shape[2]] = map
    chunk_list = list()
    start_point = box_size - stride
    cur_x, cur_y, cur_z = start_point, start_point, start_point
    while (cur_z + stride < map_shape[2] + box_size):
        next_chunk = padded_map[cur_x:cur_x + box_size, cur_y:cur_y + box_size, cur_z:cur_z + box_size]
        cur_x += stride
        if (cur_x + stride >= map_shape[0] + box_size):
            cur_y += stride
            cur_x = start_point # Reset X
            if (cur_y + stride  >= map_shape[1] + box_size):
                cur_z += stride
                cur_y = start_point # Reset Y
                cur_x = start_point # Reset X
        chunk_list.append(next_chunk)
    n_chunks = len(chunk_list)
    chunks = np.asarray(chunk_list, dtype=dtype)
    return chunks

In [26]:
def data_iter(mrcFolder):
    for file in os.listdir(mrcFolder):
        map, _, _, _, _ = parse_map(f"{mrcFolder}/{file}", ignorestart=False)
        maximum = np.percentile(map[map > 0], 99.999)
        map = np.where(map > 0, map / maximum, 0)
        chunks = split_map_into_overlapped_chunk(map, 60, 30)
        print(chunks.shape)


In [27]:
data_iter(depoFolder)

(1728, 60, 60, 60)
(1728, 60, 60, 60)
(2744, 60, 60, 60)
(729, 60, 60, 60)
(729, 60, 60, 60)
(729, 60, 60, 60)
(1000, 60, 60, 60)
(1331, 60, 60, 60)
(343, 60, 60, 60)


ValueError: repeated axis in transpose